In [1]:
import os
import shutil
from collections import Counter

# additional_data 폴더에 폴더로 데이터 저장, paths에 경로 추가 해주면 자동으로 전처리 해줌

# 원본 폴더 경로
# forder = "additional_data"
# paths = ["./"+forder+"/data/", "./"+forder+"/first/"]
paths=["./datasets/original",
       "./datasets/augmented/affine",
       "./datasets/augmented/brightness",
       "./datasets/augmented/contrast",
       "./datasets/augmented/gaussian_noise",
       "./datasets/augmented/hue",
       "./datasets/augmented/motion_blur",
       "./datasets/augmented/pipeline",
       ]

# 처리된 데이터를 저장할 폴더 경로
output_labels_folder = "./data/labels/"
output_images_folder = "./data/images/"

# 클래스 이름 목록 (인덱스에 따라 매칭)
class_names = [
    "red", "yellow", "green", "stop_sign", 
    "40_sign", "80_sign", "start_sign", "person"
]

# 클래스별 카운트 초기화
class_counts = {name: 0 for name in class_names}

# 새로운 폴더 생성
os.makedirs(output_labels_folder, exist_ok=True)
os.makedirs(output_images_folder, exist_ok=True)

# 데이터 처리
for path in paths:
    # 원본 폴더 경로
    labels_folder = os.path.join(path, "labels")
    images_folder = os.path.join(path, "images")

    # labels 폴더에서 모든 txt 파일을 순회
    for filename in os.listdir(labels_folder):
        if filename.endswith(".txt"):
            txt_path = os.path.join(labels_folder, filename)
            modified_lines = []
            class_counter = Counter()  # 클래스 빈도 카운터

            # txt 파일의 모든 줄 읽기
            with open(txt_path, "r") as file:
                lines = file.readlines()  # 모든 줄 읽기
                for line in lines:
                    parts = line.strip().split()
                    if parts:  # 비어 있지 않은 라인만 처리
                        # 클래스 카운트
                        class_counter[parts[0]] += 1
                        modified_lines.append(" ".join(parts))

            if not modified_lines:
                print(os.path.join(images_folder, filename.replace(".txt", ".jpg")))
                exit()
                # # 모든 줄이 삭제 대상 클래스였던 경우
                # image_path = os.path.join(images_folder, filename.replace(".txt", ".jpg"))
                # if os.path.exists(image_path):
                #     os.remove(image_path)
                # os.remove(txt_path)
                # print(f"Deleted file (all lines matched delete_classes): {txt_path}")
            else:
                # 가장 빈도가 높은 클래스 찾기
                try:
                    if class_counter:
                        most_common_class, _ = class_counter.most_common(1)[0]
                        image_class = class_names[int(most_common_class)]

                        # 클래스별 카운트를 증가시키며 파일 이름 생성
                        class_counts[image_class] += 1
                        new_image_name = f"{image_class}_{class_counts[image_class]:05d}"

                        # 텍스트 파일 복사본에 저장
                        output_txt_path = os.path.join(output_labels_folder, f"{new_image_name}.txt")
                        with open(output_txt_path, "w") as file:
                            file.write("\n".join(modified_lines) + "\n")

                        # 이미지 파일 복사 및 이름 변경
                        image_path = os.path.join(images_folder, filename.replace(".txt", ".jpg"))
                        output_image_path = os.path.join(output_images_folder, f"{new_image_name}.jpg")
                        if os.path.exists(image_path):
                            shutil.copy2(image_path, output_image_path)
                except:
                    print(os.path.join(images_folder, filename.replace(".txt", ".jpg")))

# 결과 출력
print("Class Counts by Index:", class_counts)


Class Counts by Index: {'red': 2340, 'yellow': 2066, 'green': 2264, 'stop_sign': 1326, '40_sign': 1062, '80_sign': 1365, 'start_sign': 70, 'person': 3851}


In [3]:
import os
import shutil
from collections import defaultdict
import random

# 설정
data_dir = "data"  # data 폴더 경로
image_dir = os.path.join(data_dir, "images")
label_dir = os.path.join(data_dir, "labels")

# train/valid/test 비율 설정
ratios = {"train": 0.9, "valid": 0.05, "test": 0.05}

# 출력 폴더 설정
output_dirs = {
    "train": {"images": os.path.join(data_dir, "train/images"), "labels": os.path.join(data_dir, "train/labels")},
    "valid": {"images": os.path.join(data_dir, "valid/images"), "labels": os.path.join(data_dir, "valid/labels")},
    "test": {"images": os.path.join(data_dir, "test/images"), "labels": os.path.join(data_dir, "test/labels")},
}

# 출력 폴더 생성
for split in output_dirs:
    for subdir, path in output_dirs[split].items():
        os.makedirs(path, exist_ok=True)

# 클래스별 파일 그룹화
class_files = defaultdict(list)

# 파일 이름에서 클래스 추출
for image_file in os.listdir(image_dir):
    class_name = image_file.split("_")[0]  # 파일 이름의 클래스 부분 추출
    label_file = os.path.splitext(image_file)[0] + ".txt"
    image_path = os.path.join(image_dir, image_file)
    label_path = os.path.join(label_dir, label_file)
    
    if os.path.exists(label_path):  # 레이블 파일이 있는 경우만 추가
        class_files[class_name].append((image_path, label_path))

# 클래스별 데이터 나누기
for class_name, files in class_files.items():
    random.shuffle(files)  # 파일을 랜덤 셔플
    total = len(files)
    train_count = int(total * ratios["train"])
    valid_count = int(total * ratios["valid"])
    test_count = total - train_count - valid_count  # 나머지는 test

    # 데이터 분배
    splits = {
        "train": files[:train_count],
        "valid": files[train_count:train_count + valid_count],
        "test": files[train_count + valid_count:]
    }
    
    for split, split_files in splits.items():
        for image_path, label_path in split_files:
            # 이미지 및 레이블 파일 이동
            shutil.copy(image_path, output_dirs[split]["images"])
            shutil.copy(label_path, output_dirs[split]["labels"])

# 클래스별 개수 출력
for class_name, files in class_files.items():
    print(f"Class '{class_name}': Total={len(files)}")

print(f"데이터가 {ratios["train"]}:{ratios["valid"]}:{ratios["test"]} 비율로 나뉘었습니다!")


Class '40': Total=1061
Class '80': Total=1325
Class 'green': Total=2006
Class 'person': Total=3851
Class 'red': Total=1968
Class 'start': Total=70
Class 'stop': Total=1318
Class 'yellow': Total=2059
데이터가 0.9:0.05:0.05 비율로 나뉘었습니다!
